In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [ ]:
train_a = pd.read_parquet('A/train_targets.parquet')
train_b = pd.read_parquet('B/train_targets.parquet')
train_c = pd.read_parquet('C/train_targets.parquet')

In [14]:
X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')


In [15]:
X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')



In [16]:
X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('C/X_test_estimated.parquet')


### Data Processing

**Handle NaN values**
- Look through all features and handle seperately

**Aggregate weather data to hourly averages, sums, max/min etc. depending on the relevance of the metric.**
- Ensure the timestamps between weather data and power production align.

Aggregating 15 minute frequency to hourly for location A on the most relevant features

In [29]:

aggregation_methods = {
    'clear_sky_rad:W': 'mean',
    'direct_rad:W': 'mean',
    'direct_rad_1h:J': 'last',
    'diffuse_rad:W': 'mean',
    'diffuse_rad_1h:J': 'last',
    'effective_cloud_cover:p': 'mean',
    'is_day:idx': 'max',
    'is_in_shadow:idx': 'sum',
    'sun_elevation:d': 'mean',
    'total_cloud_cover:p': 'mean'
}

X_train_observed_a_hourly = .resample('H').agg(aggregation_methods)

,time,pv_measurement
0,2019-06-02 22:00:00,0.00
1,2019-06-02 23:00:00,0.00
2,2019-06-03 00:00:00,0.00
3,2019-06-03 01:00:00,0.00
4,2019-06-03 02:00:00,19.36
...,...,...
34080,2023-04-30 19:00:00,9.02
34081,2023-04-30 20:00:00,0.00
34082,2023-04-30 21:00:00,0.00
34083,2023-04-30 22:00:00,0.00


In [28]:
highly_relevant = ['clear_sky_rad:W', 'direct_rad:W', 'direct_rad_1h:J', 'diffuse_rad:W', 'diffuse_rad_1h:J',
                   'effective_cloud_cover:p', 'is_day:idx', 'is_in_shadow:idx', 'sun_elevation:d', 'total_cloud_cover:p']

moderately_relevant = ['ceiling_height_agl:m', 'cloud_base_agl:m', 'snow_depth:cm', 'fresh_snow_1h:cm', 'fresh_snow_3h:cm',
                       'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms', 'visibility:m']
